In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fpt-ai/dataset/b6_test_data.csv
/kaggle/input/fpt-ai/dataset/b6_train_data.csv
/kaggle/input/fpt-ai/dataset/test_data.pkl
/kaggle/input/fpt-ai/dataset/train_data.pkl


In [2]:
!pip install -U bitsandbytes << dev null
!pip install evaluate << dev null

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `dev')
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.7 MB/s eta 0:00:00
  Created wheel for null: filename=null-0.6.1-py3-none-any.whl size=3208 sha256=11e0e09a219d6d5895c1111eb8d4b0df585ec2e9ac81ef1081762a488ff612d8
  Stored in directory: /root/.cache/pip/wheels/c2/3b/c4/0b8f74140e43ac34303a0e7b81480

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
import numpy as np
import torch
import evaluate
import random
from sklearn.model_selection import train_test_split

2025-04-26 10:42:11.291684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745664131.506766      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745664131.567478      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Load CSV and preprocess
df = pd.read_csv("/kaggle/input/fpt-ai/dataset/b6_train_data.csv")
train_data = df.dropna().reset_index(drop=True)
test_data = pd.read_csv("/kaggle/input/fpt-ai/dataset/b6_test_data.csv")

# Hàm để chuyển đổi chuỗi choices thành danh sách
def parse_choices(choices_str):
    try:
        if isinstance(choices_str, list):
            return choices_str

        # Sử dụng ast.literal_eval để chuyển đổi chuỗi thành danh sách
        try:
            return ast.literal_eval(choices_str)
        except:
            # Nếu không thể phân tích cú pháp, thực hiện xử lý thủ công
            choices_str = choices_str.strip('[]')
            choices = [choice.strip().strip("'").strip('"') for choice in choices_str.split(',')]
            return choices
    except:
        print(f"Lỗi xử lý choices: {choices_str}")
        return []

# Áp dụng hàm parse_choices cho cả tập huấn luyện và kiểm tra
train_data['choices'] = train_data['choices'].apply(parse_choices)
test_data['choices'] = test_data['choices'].apply(parse_choices)

# Chuẩn hóa câu trả lời (một số có "ANSWER: " ở đầu)
def normalize_answer(answer_str):
    if isinstance(answer_str, str):
        if "ANSWER: " in answer_str:
            return answer_str.split("ANSWER: ")[1]
    return answer_str

train_data['answer'] = train_data['answer'].apply(normalize_answer)

# Hiển thị một ví dụ sau khi xử lý
print("\nVí dụ sau khi xử lý:")
sample = train_data.iloc[0]
print(f"{sample['question']}")
print(f"{sample['choices']}")
print(f"{sample['answer']}")


Ví dụ sau khi xử lý:
Question: What will be output of the following code?
#include<stdio.h>
int main()
{
    printf("%d\t",sizeof(6.5));
    printf("%d\t",sizeof(90000));
    printf("%d",sizeof('A'));
    return 0;
}
['8 4 2', '8 4 2', '8 4 4', '8 4 3']
C


In [5]:
# Hàm định dạng dữ liệu cho mô hình
def format_example(row, include_answer=True):
    question = row['question']
    choices = row['choices']

    # Định dạng các lựa chọn thành A, B, C, D
    formatted_choices = '\n'.join([f"{chr(65+i)}. {choice}" for i, choice in enumerate(choices)])

    if include_answer:
        return f"""{question}

Options:
{formatted_choices}

Answer: {row['answer']}"""
    else:
        return f"""{question}

Options:
{formatted_choices}

Answer:"""

# Tạo tập dữ liệu để huấn luyện
train_data['text'] = train_data.apply(format_example, axis=1)
test_data['text'] = test_data.apply(format_example, axis=1, include_answer=False)

# Phân chia tập huấn luyện và tập validation
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

print(f"Số lượng mẫu trong tập huấn luyện: {len(train_df)}")
print(f"Số lượng mẫu trong tập validation: {len(val_df)}")

# Tạo dataset từ các pandas DataFrame
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_data)

# Hiển thị một ví dụ từ tập huấn luyện
print("\nVí dụ từ tập huấn luyện:")
print(train_dataset[0]['text'])

Số lượng mẫu trong tập huấn luyện: 3159
Số lượng mẫu trong tập validation: 790

Ví dụ từ tập huấn luyện:
Question: Write code to normalize a NumPy array arr by scaling its values to be between 0 and 1.

Options:
A. normalized_arr = (arr - arr.min()) / (arr.max() - arr.min())
B. normalized_arr = np.scale(arr)
C. normalized_arr = arr.normalize()
D. normalized_arr = (arr - arr.mean()) / arr.std()

Answer: A


In [6]:
train_data

,task_id,question,choices,answer,text
0,k10168,Question: What will be output of the following...,"[8 4 2, 8 4 2, 8 4 4, 8 4 3]",C,Question: What will be output of the following...
1,k10173,Question: What will be output of the following...,"[-4, -5, 10, 11]",A,Question: What will be output of the following...
2,k10174,Question: Match the following.\n Group 1 ...,"[P-4. Q-1, R-2, S-3, P-3, Q-1, R-4, S-2, P-3, ...",B,Question: Match the following.\n Group 1 ...
3,k10175,Question: Match the following.\nP. Regular exp...,"[P-4. Q-1, R-2, S-3, P-3, Q-1, R-4, S-2, P-3, ...",B,Question: Match the following.\nP. Regular exp...
4,k10176,Question: Which grammar rules violate the requ...,"[1 only, 1 and 3 only, 2 and 3 only, 3 and 4 o...",B,Question: Which grammar rules violate the requ...
...,...,...,...,...,...
3944,k00695,Question: Which technique is used in React Memo?,"[Hashing, Memoization, Sorting, Pipelining]",B,Question: Which technique is used in React Mem...
3945,k00696,Question: What is the correct definition of pr...,[Concept of passing data as props through inte...,A,Question: What is the correct definition of pr...
3946,k00697,Question: What is the correct syntax for arro...,"[(), =>, ()=>, >=()]",C,Question: What is the correct syntax for arro...
3947,k00698,Question: Which of the following is the correc...,"[It is a temporary storage of elements, It is ...",B,Question: Which of the following is the correc...


In [7]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    Trainer
)
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [8]:
model_id = "Qwen/Qwen2.5-Coder-0.5B-Instruct"

# Cấu hình lượng tử hóa 4-bit để giảm memory footprint
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Tải mô hình với cấu hình lượng tử hóa
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # device_map="auto",
    device_map={'':torch.cuda.current_device()},
    trust_remote_code=True
)

# Chuẩn bị mô hình cho huấn luyện 4-bit
model = prepare_model_for_kbit_training(model)

# Cấu hình LoRA để fine-tuning
lora_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "down_proj", "up_proj"]
)

# Áp dụng LoRA cho mô hình
peft_model = get_peft_model(model, lora_config)
print(f"Số lượng tham số huấn luyện: {peft_model.print_trainable_parameters()}")




# Hàm tiền xử lý dữ liệu để mã hóa
def preprocess_function(examples):
    # Cắt bớt các câu có kích thước vượt quá max_length
    max_length = 512  # Có thể điều chỉnh

    inputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=max_length,
        padding="max_length",
        return_tensors="pt",
    )

    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

# Áp dụng tiền xử lý
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

...

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

trainable params: 4,399,104 || all params: 498,431,872 || trainable%: 0.8826
Số lượng tham số huấn luyện: None


Map:   0%|          | 0/3159 [00:00<?, ? examples/s]

Map:   0%|          | 0/790 [00:00<?, ? examples/s]

In [9]:
# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir="results/code_mlu_model",
    # evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
)

# Khởi tạo trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Tiến hành huấn luyện mô hình
print("Bắt đầu huấn luyện mô hình...")
trainer.train()

# Lưu mô hình đã huấn luyện
peft_model.save_pretrained("results/code_mlu_model_final")
tokenizer.save_pretrained("results/code_mlu_model_final")

/tmp/ipykernel_19/2603966587.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Bắt đầu huấn luyện mô hình...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
500,0.177200
1000,0.124800
1500,0.107400
2000,0.091500
2500,0.076400
3000,0.066000
3500,0.058000
4000,0.046000
4500,0.040300
5000,0.035100


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

('results/code_mlu_model_final/tokenizer_config.json',
 'results/code_mlu_model_final/special_tokens_map.json',
 'results/code_mlu_model_final/vocab.json',
 'results/code_mlu_model_final/merges.txt',
 'results/code_mlu_model_final/added_tokens.json',
 'results/code_mlu_model_final/tokenizer.json')

In [10]:
# Tạo hàm dự đoán cho một câu hỏi
def predict_answer(question_text, model, tokenizer):
    # Chuẩn bị đầu vào
    inputs = tokenizer(
        question_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    # Tạo dự đoán
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=5,
        temperature=0.1,
        top_p=0.95,
        top_k=40,
        return_dict_in_generate=True,
        output_scores=True,
    )

    # Giải mã kết quả
    generated = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    # Trích xuất đáp án (A, B, C, D)
    answer_part = generated.split("Answer:")[-1].strip()

    # Trả về chữ cái đầu tiên (A, B, C, D) nếu có
    for letter in ["A", "B", "C", "D"]:
        if letter in answer_part:
            return letter

    # Nếu không tìm thấy chữ cái rõ ràng, trả về ký tự đầu tiên
    return answer_part[0] if answer_part else "A"

In [11]:
# Dự đoán cho tập kiểm tra
print("Dự đoán cho tập kiểm tra...")
predictions = []

# Tạo thanh tiến trình
from tqdm import tqdm

# Dự đoán từng câu hỏi trong tập kiểm tra
for example in tqdm(test_data.itertuples(), total=len(test_data)):
    task_id = example.task_id
    question_text = example.text
    prediction = predict_answer(question_text, peft_model, tokenizer)
    predictions.append({"task_id": task_id, "answer": prediction})

# Tạo DataFrame và lưu kết quả
submission_df = pd.DataFrame(predictions)
submission_df.to_csv("submission.csv", index=False)

print(f"\nĐã lưu kết quả vào file 'submission.csv' với {len(submission_df)} dự đoán")
print("\nMẫu kết quả dự đoán:")
print(submission_df.head(10))

Dự đoán cho tập kiểm tra...


  0%|          | 0/1253 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
DynamicCache + torch.export is tested on torch 2.6.0+ and may not work on earlier versions.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 1253/1253 [04:41<00:00,  4.45it/s]


Đã lưu kết quả vào file 'submission.csv' với 1253 dự đoán

Mẫu kết quả dự đoán:
  task_id answer
0  k10171      C
1  k10182      D
2  k10184      A
3  k10206      D
4  k10215      B
5  k10231      A
6  k10234      D
7  k10238      C
8  k10248      B
9  k10249      B


## Test old version

In [12]:
# Tải mô hình với cấu hình lượng tử hóa
model_old = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # device_map="auto",
    device_map={'':torch.cuda.current_device()},
    trust_remote_code=True
)

In [13]:
# Dự đoán cho tập kiểm tra
print("Dự đoán cho tập kiểm tra...")
predictions = []

# Tạo thanh tiến trình
from tqdm import tqdm

# Dự đoán từng câu hỏi trong tập kiểm tra
for example in tqdm(test_data.itertuples(), total=len(test_data)):
    task_id = example.task_id
    question_text = example.text
    prediction = predict_answer(question_text, model_old, tokenizer)
    predictions.append({"task_id": task_id, "answer": prediction})

# Tạo DataFrame và lưu kết quả
submission_df = pd.DataFrame(predictions)
submission_df.to_csv("submission_old.csv", index=False)

print(f"\nĐã lưu kết quả vào file 'submission.csv' với {len(submission_df)} dự đoán")
print("\nMẫu kết quả dự đoán:")
print(submission_df.head(10))

Dự đoán cho tập kiểm tra...


100%|██████████| 1253/1253 [04:37<00:00,  4.51it/s]


Đã lưu kết quả vào file 'submission.csv' với 1253 dự đoán

Mẫu kết quả dự đoán:
  task_id answer
0  k10171      B
1  k10182      A
2  k10184      B
3  k10206      B
4  k10215      A
5  k10231      C
6  k10234      A
7  k10238      B
8  k10248      C
9  k10249      B
